OBS: Algumas partes foram comentadas para poder rodar sem a presença do arduino
--> Buscar essas partes por "BACALHAU"

IMPORTAÇÃO DE BIBLIOTECAS

In [ ]:
import cv2
import mediapipe as mp
import math
import pandas as pd
from datetime import datetime 
import serial
import time



INICIALIZAÇÃO DO ARDUINO

In [9]:
#arduino = serial.Serial(port='COM5', baudrate=9600, timeout=1)  
#BACALHAU

## FUNÇÕES

FUNÇÕES PARA CALCULAR ANGULO

Calculo do angulo em relação a uma reta

In [10]:
def calcula_angulo(a_x: float, a_y: float, b_x: float, b_y: float):
    '''
    Função que calcula angulo entre um vetor e um vetor vertical a partir de 2 pontos.

    Calculo efetuado a partir da seguinte fórmula: cos θ = (u.v)/(|u||v|)

    O ponto (a) deve ser o ponto mais baixo, que se mexe em relação a referência
    O ponto (b) deve ser o ponto de origem (o ponto mais alto, que não mexe)

    Exemplo: Ponto (a) é o mão direita (ponto de refêrencia móvel) e ponto (b) é o ombro direito (ponto imóvel)
    '''

    # Calculo dos vetores
    vetor_ab = (b_x - a_x, b_y - a_y) # Tupla com coordenadas do vetor
    vetor_vertical = (0,-1) # Vetor (0,-1) pois preciso pegar o ângulo entre a reta (passando pelos dois pontos) e uma reta vertical

    # Calculo dos componentes da formula
    produto_escalar = vetor_ab[0] * vetor_vertical[0] + vetor_ab[1] * vetor_vertical[1] # (u.v)
    norma_ab = math.sqrt(vetor_ab[0]**2 + vetor_ab[1]**2) # (|u|)
    norma_v_vertical = math.sqrt(vetor_vertical[0]**2 + vetor_vertical[1]**2) # (|v|)

    cos_angulo = produto_escalar / (norma_ab * norma_v_vertical) # cos θ 
    angulo_radianos = math.acos(cos_angulo) # arcos(cos(θ))
    angulo_graus = math.degrees(angulo_radianos) # Porque a função acos retorna em radiano

    return round(angulo_graus, 2) # Retorna apenas duas casas decimais

FUNÇÃO QUE POSCIONA LEGENDA NA TELA

In [11]:
def funcao_texto(texto, cor, img, coord1,coord2):
   '''
   Função que posiciona a legenda na tela de exibição do mediapipe.

   Argumentos da função:
   texto => texto que será exibido na tela
   cor => cor do retangulo que ficará atrás do texto
   img => próprio da função
   coord1 e coord2 => define posicionamento do texto
   '''

   # Definição das propriedades do texto
   font = cv2.FONT_HERSHEY_SIMPLEX
   font_scale = 0.7 
   thickness = 2

   # Obtendo o tamanho do texto
   (text_width, text_height), baseline = cv2.getTextSize(texto, font, font_scale, thickness)

   # Coordenadas do canto superior esquerdo
   x, y = coord1, coord2

   # Calculando o retângulo com base no tamanho do texto
   top_left = (x, y - text_height - baseline)
   bottom_right = (x + text_width, y + baseline)

   # Desenhando o retângulo
   cv2.rectangle(img, top_left, bottom_right, cor, -1)

   cv2.putText(img, texto, (x, y), font, font_scale, (255, 255, 255), thickness)


FUNÇÃO ENVIA STRING PARA ARDUINO

In [12]:
def envia_string(string):
    arduino.write(f"{'Movimento ' + string}\n".encode())  
        # Função write -> Envia valor ao Arduino em bytes
        # Função encode -> transforma em bytes
    return 

## DICIONÁRIOS

Dicionário dos braços

In [13]:
dicionario_poses = {
    "cristo redentor": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (90 - 30) <= anguloBEC <= (90 + 30) and
            (90 - 30) <= anguloBEB <= (90 + 30) and
            (90 - 30) <= anguloBDC <= (90 + 30) and
            (90 - 30) <= anguloBDB <= (90 + 30)
        )
    },
    "maos pro alto": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (180 - 30) <= anguloBDC <= (180 + 30) and
            (180 - 30) <= anguloBDB <= (180 + 30) and
            (180 - 30) <= anguloBEC <= (180 + 30) and
            (180 - 30) <= anguloBEB <= (180 + 30)
        )
    },
    "maos pra baixo": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (0 - 30) <= anguloBDC <= (0 + 30) and
            (0 - 30) <= anguloBDB <= (0 + 30) and
            (0 - 30) <= anguloBEC <= (0 + 30) and
            (0 - 30) <= anguloBEB <= (0 + 30)
        )
    },
    "esquerda cima direita baixo": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (180 - 30) <= anguloBEC <= (180 + 30) and
            (180 - 30) <= anguloBEB <= (180 + 30) and
            (0 - 30) <= anguloBDC <= (0 + 30) and
            (0 - 30) <= anguloBDB <= (0 + 30)
        )
    },
    "direita cima esquerda baixo": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (180 - 30) <= anguloBDC <= (180 + 30) and
            (180 - 30) <= anguloBDB <= (180 + 30) and
            (0 - 30) <= anguloBEC <= (0 + 30) and
            (0 - 30) <= anguloBEB <= (0 + 30)
        )
    },
    "formato U":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (90 - 30) <= anguloBDC <= (90 + 30) and
            (180 - 30) <= anguloBDB <= (180 + 30) and
            (90 - 30) <= anguloBEC <= (90 + 30) and
            (180 - 30) <= anguloBEB <= (180 + 30)
        )
    },
    "formato U invertido":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (90 - 30) <= anguloBDC <= (90 + 30) and
            (0 - 30) <= anguloBDB <= (0 + 30) and
            (90 - 30) <= anguloBEC <= (90 + 30) and
            (0 - 30) <= anguloBEB <= (0 + 30)
        )
    },
    "formato S":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (90 - 30) <= anguloBDC <= (90 + 30) and
            (0 - 30) <= anguloBDB <= (0 + 30) and
            (90 - 30) <= anguloBEC <= (90 + 30) and
            (180 - 30) <= anguloBEB <= (180 + 30)
        )
    },
    "formato Ƨ":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (90 - 30) <= anguloBEC <= (90 + 30) and
            (0 - 30) <= anguloBEB <= (0 + 30) and
            (90 - 30) <= anguloBDC <= (90 + 30) and
            (180 - 30) <= anguloBDB <= (180 + 30)
        )
    },
    "disco move":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (45 - 30) <= anguloBEC <= (45 + 30) and
            (45 - 30) <= anguloBEB <= (45 + 30) and
            (135 - 30) <= anguloBDC <= (135 + 30) and
            (135 - 30) <= anguloBDB <= (135 + 30)
        )
    },
    "disco-disco move":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (45 - 30) <= anguloBDC <= (45 + 30) and
            (45 - 30) <= anguloBDB <= (45 + 30) and
            (135 - 30) <= anguloBEC <= (135 + 30) and
            (135 - 30) <= anguloBEB <= (135 + 30)
        )
    },
    "uzinho":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (50 - 30) <= anguloBDC <= (50 + 30) and
            (180 - 30) <= anguloBDB <= (180 + 30) and
            (50 - 30) <= anguloBEC <= (50 + 30) and
            (180 - 30) <= anguloBEB <= (180 + 30)
        )
    },
    "v":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (135 - 30) <= anguloBDC <= (135 + 30) and
            (135 - 30) <= anguloBDB <= (135 + 30) and
            (135 - 30) <= anguloBEC <= (135 + 30) and
            (135 - 30) <= anguloBEB <= (135 + 30)
        )
    },
     "maos na cabeca":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (90 - 30) <= anguloBDC <= (90 + 30) and
            (130 - 30) <= anguloBDB <= (130 + 30) and
            (90 - 30) <= anguloBEC <= (90 + 30) and
            (135 - 30) <= anguloBEB <= (130 + 30)
        )
    }, 
}

Dicionário das pernas

In [14]:
dicionario_poses_pernas = {
    "retas": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (0 - 30) <= anguloPEC <= (0 + 30) and
            (0 - 30) <= anguloPEB <= (0 + 30) and
            (0 - 30) <= anguloPDC <= (0 + 30) and
            (0 - 30) <= anguloPDB <= (0 + 30)
        )
    },
    "dobradas": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (90 - 30) <= anguloPEC <= (90 + 30) and
            (0 - 30) <= anguloPEB <= (0 + 30) and
            (90 - 30) <= anguloPDC <= (90 + 30) and
            (0 - 30) <= anguloPDB <= (0 + 30)
        )
    },
    "chutes": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (90 - 30) <= anguloPEC <= (90 + 30) and
            (90 - 30) <= anguloPEB <= (90 + 30) and
            (90 - 30) <= anguloPDC <= (90 + 30) and
            (90 - 30) <= anguloPDB <= (90 + 30)
        )
    },
    "chuta esquerdo": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (90 - 30) <= anguloPEC <= (90 + 30) and
            (90 - 30) <= anguloPEB <= (90 + 30) and
            (0 - 30) <= anguloPDC <= (90 + 30) and
            (0 - 30) <= anguloPDB <= (90 + 30)
        )
    },
    "chuta direito": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (0 - 30) <= anguloPEC <= (90 + 30) and
            (0 - 30) <= anguloPEB <= (90 + 30) and
            (90 - 30) <= anguloPDC <= (90 + 30) and
            (90 - 30) <= anguloPDB <= (90 + 30)
        )
    },
    "dobra direita": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (90 - 30) <= anguloPEC <= (90 + 30) and
            (0 - 30) <= anguloPEB <= (0 + 30) and
            (0 - 30) <= anguloPDC <= (90 + 30) and
            (0 - 30) <= anguloPDB <= (0 + 30)
        )
    },
    "dobra esquerda": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (0 - 30) <= anguloPEC <= (90 + 30) and
            (0 - 30) <= anguloPEB <= (0 + 30) and
            (90 - 30) <= anguloPDC <= (90 + 30) and
            (0 - 30) <= anguloPDB <= (0 + 30)
        )
    },
}

CÓDIGO PRINCIPAL MIDIAPIPE

In [15]:
# Inicializa a captura de vídeo em tempo real da webcam
video = cv2.VideoCapture(0) # Se não funcionar, trocar para 1 (depende do computador e quantas cameras tem)

# Verifica se a câmera foi inicializada corretamente
if not video.isOpened():
    print("Erro: Não foi possível acessar a câmera.")
    exit()

# Inicializa o MediaPipe Pose
mp_pose = mp.solutions.pose # Importa a biblioteca dos landmarks
pose = mp_pose.Pose(min_tracking_confidence=0.5, min_detection_confidence=0.5) # Define parâmetros
mp_draw = mp.solutions.drawing_utils # Importa a biblioteca de desenho


while True:
    success, img = video.read()

    # Verifica se a captura foi bem-sucedida e se a imagem não é None
    if not success or img is None:
        print("Erro: Não foi possível capturar a imagem da câmera.")
        break

    # Redimensiona a imagem para garantir que seja processada corretamente
    img = cv2.resize(img, (640, 480))

    # Converte a imagem para RGB para o MediaPipe
    videoRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Processa a imagem com MediaPipe Pose
    results = pose.process(videoRGB)

    # Obtém os landmarks do corpo detectado
    if results.pose_landmarks:
        
        # Desenha os landmarks na imagem
        mp_draw.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS) 

        h, w, _ = img.shape
        landmarks = results.pose_landmarks.landmark

        #-----------------------------------------------------------------------------------------------------------------------------------

        # Calcula as coordenadas dos pontos específicos

        # Pé Direito
        peDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y * h)
        peDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].x * w)
        peDZ = int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].z)

        # Pé Esquerdo
        peEY = int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y * h)
        peEX = int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x * w)
        peEZ = int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].z)

        # Mão direita
        maoDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_INDEX].y * h)
        maoDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_INDEX].x * w)
        maoDZ = int(landmarks[mp_pose.PoseLandmark.RIGHT_INDEX].z)
        # Mão esquerda
        maoEY = int(landmarks[mp_pose.PoseLandmark.LEFT_INDEX].y * h)
        maoEX = int(landmarks[mp_pose.PoseLandmark.LEFT_INDEX].x * w)
        maoEZ = int(landmarks[mp_pose.PoseLandmark.LEFT_INDEX].z)


        # Torso direito
        torsoDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x * w)
        torsoDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y * h)

        
        # Torso Esquerdo
        torsoEX = int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * w)
        torsoEY = int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * h)

        # Ombro Direito
        ombroDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w)
        ombroDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h)

        # Ombro Esquerdo
        ombroEX = int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w)
        ombroEY = int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h)

        # Cotovelo Direito
        cotoveloDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x * w)
        cotoveloDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y * h)

        # Cotovelo Esquerdo
        cotoveloEX = int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x * w)
        cotoveloEY = int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y * h)

        # Joelho Direito
        joelhoDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].x * w)
        joelhoDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y * h)

        # Joelho Esquerdo
        joelhoEX = int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x * w)
        joelhoEY = int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y * h)


        #-----------------------------------------------------------------------------------------------------------------------------------


        # Calcula os ângulos e exibe (CALCULO DE ANGULO APENAS PARA EXIBIÇÃO: Entre vetor e vetor vertical)
        
        
        # Primeira linha: calcula angulo
        # Segunda linha: exibe angulo

        # Perna direita (baixo)
        anguloPDB = calcula_angulo(peDX,peDY,joelhoDX,joelhoDY)
        cv2.putText(img, str(anguloPDB), (int((peDX+joelhoDX)/2)-40, int((peDY+joelhoDY)/2)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Perna direita (cima)
        anguloPDC = calcula_angulo(joelhoDX,joelhoDY,torsoDX,torsoDY)
        cv2.putText(img, str(anguloPDC), (int((joelhoDX+torsoDX)/2)-40, int((joelhoDY+torsoDY)/2)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

         # Perna esquerda (baixo)
        anguloPEB = calcula_angulo(peEX,peEY,joelhoEX,joelhoEY)
        cv2.putText(img, str(anguloPEB), (int((peEX+joelhoEX)/2)+40, int((peEY+joelhoEY)/2)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        # Perna esquerda (cima)
        anguloPEC = calcula_angulo(joelhoEX,joelhoEY,torsoEX,torsoEY)
        cv2.putText(img, str(anguloPEC), (int((joelhoEX+torsoEX)/2)+40, int((joelhoEY+torsoEY)/2)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)


        # Braço (cima) direito
        anguloBDC = calcula_angulo(cotoveloDX,cotoveloDY,ombroDX,ombroDY)
        cv2.putText(img, str(anguloBDC), (int((cotoveloDX+ombroDX)/2)-40, int((cotoveloDY+ombroDY)/2)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        # Braço (baixo) direito
        anguloBDB = calcula_angulo(maoDX,maoDY,cotoveloDX,cotoveloDY)
        cv2.putText(img, str(anguloBDB), (int((cotoveloDX + maoDX) / 2)-40, int((cotoveloDY + maoDY) / 2)), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        

        # Braço (cima) esquerdo
        anguloBEC = calcula_angulo(cotoveloEX,cotoveloEY,ombroEX,ombroEY)
        cv2.putText(img, str(anguloBEC), (int((cotoveloEX+ombroEX)/2)+40, int((cotoveloEY+ombroEY)/2)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        # Braço (baixo) esquerdo
        anguloBEB = calcula_angulo(maoEX,maoEY,cotoveloEX,cotoveloEY)
        cv2.putText(img, str(anguloBEB), (int((cotoveloEX + maoEX) / 2)+40, int((cotoveloEY + maoEY) / 2)), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        
        
        #-----------------------------------------------------------------------------------------------------------------------------------


        # Exibição visual

        texto = str(peDX) + "," + str(peDY) + ","+ str(peDY) 
        cv2.putText(img, texto, (peDX,peDY), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        
        texto = str(maoDX) + "," + str(maoDY) + ","+ str(maoDZ) 
        cv2.putText(img, texto, (maoDX,maoDY), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        

        #-----------------------------------------------------------------------------------------------------------------------------------


        
        #-----------------------------------------------------------------------------------------------------------------------------------

        # Percorrer o dicionário e ver se a pessoa está fazendo uma das poses dos BRAÇOS

        for movimento, angulos in dicionario_poses.items():
            if angulos["condicao"](anguloBEC, anguloBEB, anguloBDC, anguloBDB):
                cor = (255, 105, 180)
                texto = movimento
                coord1 = 10
                coord2= 300
                funcao_texto(texto, cor, img, coord1, coord2)
                #envia_string("Movimento " + texto) #OBS: Vai ter que diferencia movimento de perna e braço, tem que ver o codigo do Bruno
                #BACALHAU

        #-----------------------------------------------------------------------------------------------------------------------------------


        # Percorrer o dicionário e ver se a pessoa está fazendo uma das poses das PERNAS

        for movimento, angulos in dicionario_poses_pernas.items():
            if angulos["condicao"](anguloPEC, anguloPEB, anguloPDC, anguloPDB):
                cor = (255, 105, 180)
                texto = movimento
                coord1 = 10
                coord2= 250
                funcao_texto(texto, cor, img, coord1, coord2)
                #envia_string("Movimento " + texto) # Adicionei a string movimento para ser coerente com o código do Bruno    
                #BACALHAU


#-----------------------------------------------------------------------------------------------------------------------------------

    # Exibição do vídeo em tempo real com os landmarks
    cv2.imshow('Resultado', img)

    # Verifica se a tecla 'q' foi pressionada para sair do loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libera os recursos
video.release()
cv2.destroyAllWindows()
